In [1]:
# import logging
# import sys

# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [1]:
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.faiss import FaissVectorStore
from IPython.display import Markdown, display

In [2]:
import os
import openai

# needed to synthesize responses later
os.environ["OPENAI_API_KEY"] = "sk-..."
openai.api_key = os.environ["OPENAI_API_KEY"]

# Documents

In [3]:
# load documents
documents = SimpleDirectoryReader("/home/surya/NEU/CS5100 FAI/Project/ResearchLens/uploads").load_data()

In [4]:
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=10)

In [5]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# loads BAAI/bge-small-en
# embed_model = HuggingFaceEmbedding()

# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")

In [6]:
from llama_index.core import Settings

Settings.embed_model = embed_model
Settings.text_splitter = text_splitter
Settings.llm = None

LLM is explicitly disabled. Using MockLLM.


In [7]:
index = VectorStoreIndex.from_documents(
    documents, embed_model=embed_model, transformations=[text_splitter]
)

disk away

In [9]:
index.storage_context.persist("./storage")

In [10]:
# # save index to disk
# index.storage_context.persist("./storage")

# storage_context = StorageContext.from_defaults(
#     persist_dir="./storage"
# )
# index = load_index_from_storage(storage_context=storage_context)

## Test a query

In [11]:
query_engine = index.as_query_engine()

In [12]:
response = query_engine.query("""
Table 1: Object tracking datasets comparison. In addition to larger scale than previous datasets,
the scenarios captured by EgoTracks represent a significantly harder challenge for SOTA trackers,
suggesting room for improved tracking methodology.
SOTA
Dataset
 Video Hours
 Avg. Length (s)
 Ann. FPS
 Ann. Type
 Egocentric
(P/AO)∗
ImageNet-Vid [62]
 15.6
 10.6
 25
 mask
 No
YT-VOS [78]
 5.8
 4.6
 5
 mask
 No
 -/83.6 [31]
DAVIS 17 [61]
 0.125
 3
 24
 mask
 No
 -/86.3 [7]
TAO [15]
 29.7
 36.8
 1
 mask
 No
UVO [74]
 2.8
 10
 30
 mask
 No
 -/73.7 [58]
EPIC-KITCHENS
[14]
 36
 12∗∗
 0.9
 mask
 Yes
 -/74.2 [58]
VISOR
GOT-10k [32]
 32.8
 12.2
 10
 bbox
 No
 -/75.6 [9]
OxUvA [68]
 14.4
 141.2
 1
 bbox
 No
LaSOT [21]
 31.92
 82.1
 30
 bbox
 No
 80.3/- [9]
TrackingNet [57]
 125.1
 14.7
 28
 bbox
 No
 86/- [9]
TREK-150 [19, 20]
 0.45
 10.81
 60
 bbox
 Yes
 -/50.5 [19, 20]
EgoTracks (Ours)
 602.9
 367.9
 5
 bbox
 Yes
 45/54.1
∗
 : P: Precision, AO: average overlap (J-Score for mask-based datasets). ∗∗: Original video is 720s.
""".strip())